In [ ]:
import segmentation_models_pytorch as smp
import torch
import os
import cv2
import albumentations as A
from skimage import io

In [ ]:
transform = A.Compose([
    A.Resize(width=256, height=256)
])

In [ ]:
model = smp.Unet(encoder_name = "timm-resnest50d", classes = 2, in_channels = 3)
model.eval()

In [ ]:
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\u-timm-resnest50d.pt"))

In [ ]:
image_dir = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\Cells\test\images"

In [ ]:
for cnt, im in enumerate(os.listdir(image_dir)):
    im = io.imread(os.path.join(image_dir, im))
    im = im/255.
    im = transform(image = im)["image"]
    im = torch.tensor(im, dtype = torch.float).permute(2, 0, 1).unsqueeze(0)
    print(im.shape)
    pred = model(im)
    pred = torch.argmax(pred, 1)[0] * 255.
    cv2.imwrite(str(cnt) + ".png", pred.numpy())

In [ ]:
ii = cv2.imread(r"C:\Users\phili\Downloads\Telegram Desktop\Cells1\Cells\train\labels\05__1_4685_10225.png",0)

In [ ]:
ii = ii * 255.

In [ ]:
cv2.imwrite("test.png", ii)